In [54]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    StockBarsRequest,
    StockTradesRequest,
    StockQuotesRequest
)
from alpaca.trading.requests import (
    GetAssetsRequest, 
    MarketOrderRequest, 
    LimitOrderRequest, 
    StopOrderRequest, 
    StopLimitOrderRequest, 
    TakeProfitRequest, 
    StopLossRequest, 
    TrailingStopOrderRequest, 
    GetOrdersRequest, 
    ClosePositionRequest
)
from alpaca.trading.enums import ( 
    AssetStatus, 
    AssetExchange, 
    OrderSide, 
    OrderType, 
    TimeInForce, 
    OrderClass, 
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

from sklearn.preprocessing import StandardScaler

In [2]:
api_key = ""
secret_key = ""

paper = True 

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

# Fetching Market Data

In [44]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [45]:
symbols = ['META', 'AAPL', 'MSFT', 'AMZN', 'GOOG', 'TSLA']
now = datetime.now(ZoneInfo("America/New_York"))
start_date = datetime(2022, 9, 1)  # Starting from September 1, 2022
end_date = datetime(2023, 3, 1)    # Ending on September 1, 2023

In [60]:
market_data_dict = {}
for symbol in symbols:
    print(f"Fetching {symbol} data...")
    req = StockBarsRequest(
        symbol_or_symbols = [symbol],
        timeframe=TimeFrame(amount = 5, unit = TimeFrameUnit.Minute), 
        start = start_date,     
        end=end_date,                                                        
    )
    market_data_dict[symbol] = stock_historical_data_client.get_stock_bars(req).df.loc[:,["close","volume"]]

Fetching META data...
Fetching AAPL data...
Fetching MSFT data...
Fetching AMZN data...
Fetching GOOG data...
Fetching TSLA data...


In [64]:
market_data_dict['GOOG']

close  volume        bb       sma
symbol timestamp                                                    
GOOG   2022-09-01 08:00:00+00:00  108.40  4977.0       NaN       NaN
       2022-09-01 08:05:00+00:00  108.62   703.0       NaN       NaN
       2022-09-01 08:10:00+00:00  108.41   672.0       NaN       NaN
       2022-09-01 08:15:00+00:00  108.53   652.0       NaN       NaN
       2022-09-01 08:20:00+00:00  108.50  2147.0       NaN       NaN
...                                  ...     ...       ...       ...
       2023-02-28 23:20:00+00:00   90.02   302.0 -0.350084 -0.000588
       2023-02-28 23:40:00+00:00   90.02   401.0 -0.333014 -0.000566
       2023-02-28 23:45:00+00:00   90.07   295.0  0.006611  0.000011
       2023-02-28 23:50:00+00:00   89.94  5694.0 -0.756311 -0.001366
       2023-02-28 23:55:00+00:00   90.08   336.0  0.111203  0.000200

[19604 rows x 4 columns]

In [65]:
market_data_dict['GOOG']['sma'].max()

np.float64(0.038333017916611256)

# Technical Indicator Functions

In [68]:
def calculate_bollinger_bands(data, window=20, num_of_std=2):
    """Calculate Bollinger Bands ratio wrt current price"""
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    bb_ratio = (data - rolling_mean) / (rolling_std * num_of_std)
    return bb_ratio


In [67]:
def calculate_rsi(data, window=20):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [62]:
def calculate_sma(data, window=20):
    """Calculate SMA ratio of current price."""
    rolling_mean = data.rolling(window=window).mean()
    sma = (data / rolling_mean) - 1
    return sma

# Add TI Features

In [69]:
for symbol in symbols:
    # Calc new indicators
    market_data_dict[symbol]['bb'] = calculate_bollinger_bands(market_data_dict[symbol]['close'])
    market_data_dict[symbol]['sma'] = calculate_sma(market_data_dict[symbol]['close'])
    market_data_dict[symbol]['rsi'] = calculate_rsi(market_data_dict[symbol]['close'])
    # TODO: Remove NaN rows
    # TODO: Generate Labels
    # TODO: Stack all dataframes
    # TODO: Normalize

In [70]:
market_data_dict['GOOG']

close  volume        bb       sma  \
symbol timestamp                                                       
GOOG   2022-09-01 08:00:00+00:00  108.40  4977.0       NaN       NaN   
       2022-09-01 08:05:00+00:00  108.62   703.0       NaN       NaN   
       2022-09-01 08:10:00+00:00  108.41   672.0       NaN       NaN   
       2022-09-01 08:15:00+00:00  108.53   652.0       NaN       NaN   
       2022-09-01 08:20:00+00:00  108.50  2147.0       NaN       NaN   
...                                  ...     ...       ...       ...   
       2023-02-28 23:20:00+00:00   90.02   302.0 -0.350084 -0.000588   
       2023-02-28 23:40:00+00:00   90.02   401.0 -0.333014 -0.000566   
       2023-02-28 23:45:00+00:00   90.07   295.0  0.006611  0.000011   
       2023-02-28 23:50:00+00:00   89.94  5694.0 -0.756311 -0.001366   
       2023-02-28 23:55:00+00:00   90.08   336.0  0.111203  0.000200   

                                         rsi  
symbol timestamp                              
GOOG   2022-09-01 08:00:00+00:00         NaN  
       2022-09-01 08:05:00+00:00  100.000000  
       2022-09-01 08:10:00+00:00   51.162791  
       2022-09-01 08:15:00+00:00   61.818182  
       2022-09-01 08:20:00+00:00   58.620690  
...                                      ...  
       2023-02-28 23:20:00+00:00   49.206349  
       2023-02-28 23:40:00+00:00   48.391259  
       2023-02-28 23:45:00+00:00   48.387097  
       2023-02-28 23:50:00+00:00   45.454545  
       2023-02-28 23:55:00+00:00   49.295775  

[19604 rows x 5 columns]